# Creating a date dimension table

In [1]:
# importing data analysis library
import pandas as pd

# first column
# creating the base range
date_range = pd.date_range(start= '1970-01-01',
                          end= '2030-12-31')
# range as df with format dd-mm-yyyy
date_dim = pd.DataFrame(data= date_range,
                        columns= ['date'])
date_dim['date'] = date_dim['date'].dt.strftime('%d-%m-%Y')

# displaying the df so far
date_dim

,date
0,01-01-1970
1,02-01-1970
2,03-01-1970
3,04-01-1970
4,05-01-1970
...,...
22275,27-12-2030
22276,28-12-2030
22277,29-12-2030
22278,30-12-2030


In [2]:
# extracting basic features from the date
date_dim['year'] = date_dim['date'].apply(lambda x: 
                                          pd.to_datetime(x, 
                                          format= '%d-%m-%Y')
                                          .year)
date_dim['quarter'] = date_dim['date'].apply(lambda x:
                                             'Q' + str(pd.to_datetime(
                                             x, format= '%d-%m-%Y')
                                                       .quarter))
date_dim['month'] = date_dim['date'].apply(lambda x:
                                           pd.to_datetime(x,
                                           format= '%d-%m-%Y')
                                           .month)
date_dim['monthName'] = date_dim['date'].apply(lambda x:
                                               pd.to_datetime(
                                                x, format= '%d-%m-%Y')
                                               .strftime('%B'))
date_dim['year-quarter'] = date_dim.apply(lambda x: 
                              f"{x['year']}-{x['quarter']}", axis= 1)
date_dim['year-month'] = date_dim.apply(lambda x:
                              f"{x['monthName']} {x['year']}", axis= 1)
date_dim['day_of_week'] = date_dim['date'].apply(lambda x:
                              pd.to_datetime(x, format= '%d-%m-%Y')
                                                  .strftime('%A'))
date_dim['day#_of_week'] = date_dim['date'].apply(lambda x:
                              pd.to_datetime(x, format= '%d-%m-%Y')
                                                 .dayofweek + 1)
date_dim['day_of_month'] = date_dim['date'].apply(lambda x:
                                           pd.to_datetime(x,
                                           format= '%d-%m-%Y')
                                           .day)
date_dim['day_of_year'] = date_dim['date'].apply(lambda x:
                                           pd.to_datetime(x,
                                           format= '%d-%m-%Y')
                                           .dayofyear)
date_dim['week_of_year'] = date_dim['date'].apply(lambda x:
                                           pd.to_datetime(x,
                                           format= '%d-%m-%Y')
                                           .isocalendar()[1])

date_dim.sample(5)

,date,year,quarter,month,monthName,year-quarter,year-month,day_of_week,day#_of_week,day_of_month,day_of_year,week_of_year
5687,28-07-1985,1985,Q3,7,July,1985-Q3,July 1985,Sunday,7,28,209,30
8362,23-11-1992,1992,Q4,11,November,1992-Q4,November 1992,Monday,1,23,328,48
6845,28-09-1988,1988,Q3,9,September,1988-Q3,September 1988,Wednesday,3,28,272,39
3877,13-08-1980,1980,Q3,8,August,1980-Q3,August 1980,Wednesday,3,13,226,33
5521,12-02-1985,1985,Q1,2,February,1985-Q1,February 1985,Tuesday,2,12,43,7


In [3]:
# extracting advanced features
date_dim['is_weekend'] = date_dim['day#_of_week'].apply(lambda x:
                                                        x >= 6)
date_dim['is_leap_year'] = date_dim['year'].apply(lambda x: 
                                                  pd.to_datetime(
                                                  f"{x}-01-01"
                                                  ).is_leap_year)
date_dim['epoch_time'] = date_dim['date'].apply(lambda x: 
                                                int(pd.to_datetime(
                                                x, format='%d-%m-%Y'
                                                ).timestamp()))
date_dim['julian_date'] = date_dim['date'].apply(lambda x: 
                                                 pd.to_datetime(
                                                 x, format='%d-%m-%Y'
                                                 ).to_julian_date())

# extracting seasons
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'

date_dim['season'] = date_dim['date'].apply(lambda x: 
                                            get_season(
                                            pd.to_datetime(
                                            x, format='%d-%m-%Y')))

# extracting fiscal information
date_dim['fiscal_year'] = date_dim['date'].apply(lambda x: 
                             pd.to_datetime(x, format='%d-%m-%Y').year 
                            if pd.to_datetime(x, format='%d-%m-%Y').month >= 4 
                            else pd.to_datetime(x, format='%d-%m-%Y').year - 1)
date_dim['fiscal_quarter'] = date_dim['date'].apply(lambda x: 
                            'Q' + str(((pd.to_datetime(
                            x, format='%d-%m-%Y').month - 4) % 12) // 3 + 1))

date_dim.sample(5)

,date,year,quarter,month,monthName,year-quarter,year-month,day_of_week,day#_of_week,day_of_month,day_of_year,week_of_year,is_weekend,is_leap_year,epoch_time,julian_date,season,fiscal_year,fiscal_quarter
10010,29-05-1997,1997,Q2,5,May,1997-Q2,May 1997,Thursday,4,29,149,22,False,False,864864000,2450597.5,Spring,1997,Q1
9185,24-02-1995,1995,Q1,2,February,1995-Q1,February 1995,Friday,5,24,55,8,False,False,793584000,2449772.5,Winter,1994,Q4
74,16-03-1970,1970,Q1,3,March,1970-Q1,March 1970,Monday,1,16,75,12,False,False,6393600,2440661.5,Spring,1969,Q4
9560,05-03-1996,1996,Q1,3,March,1996-Q1,March 1996,Tuesday,2,5,65,10,False,True,825984000,2450147.5,Spring,1995,Q4
4040,23-01-1981,1981,Q1,1,January,1981-Q1,January 1981,Friday,5,23,23,4,False,False,349056000,2444627.5,Winter,1980,Q4


## Exporting to .csv for use in other data modelling software

In [4]:
date_dim.to_csv('date_dim.csv')